In [1]:
# Chinese Restaurant Process Clustering
# Adapted from https://github.com/attapol/crp/blob/master/crp_clustering.py
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import scipy.cluster.vq as vq
import random

In [2]:
class CRPClusterModel(object):
    """Chinese Restaurant Process Infinite Mixture Model
      Non-parametric Bayesian clustering with Chinese Restaurant Process prior
      The parameters for Gibbs sampling can be specified:
        num_iter : number of iterations to run. One iteration cycles through every data point once.
        eb_start : The trial where Empirical Bayes alpha adjustment begins
        eb_interval : The interval (number of trials) at which we adjust alpha
      """

    def __init__(self, alpha, likelihood_fn):
        """Initialize with the concentration hyperparameter alpha and likelihood function
        The likelihood function must be have this form
        def likelihood_fn(data, i, clustering, cluster_assn):
          Returns a vector x of length len(clustering) + 1 
          x[j] = P(data[i] | the cluster assignment so far AND data[i] assign to cluster j)
        where
        clustering - a list of clusters. Each cluster is a list of indices in the data
        cluster assignment - a list of cluster number (assignment)
          Examples
          Cluster 0 contains data from [1, 2, 5]
          Cluster 1 contains data from [0, 3, 4]
          Then clustering == [ [1,2,5], [0,3,4] ]
          AND cluster_assn = [1, 0, 0, 1, 1, 0]
          Note that the two formats are redundant.
        """
        self.alpha = alpha
        self.likelihood_fn = likelihood_fn

        #gibbs sampling parameters
        self.num_iter = 100
        self.eb_start = 20
        self.eb_interval = 5

    def initialize_assn(self, data):
        """Initial cluster assignment before Gibbs sampling Process
        """
        clustering = []
        cluster_assn = []

        for i in range(len(data)):
          crp_prior = [(len(x) + 0.0) / (i + self.alpha) for j, x in enumerate(clustering)]
          crp_prior.append(self.alpha / (i + self.alpha))
          crp_prior = np.array(crp_prior)
          likelihood = self.likelihood_fn(data, i, clustering, cluster_assn)
          probs = crp_prior * likelihood
          probs = probs/np.sum(probs)

          cluster = np.random.choice(len(probs), p=probs)

          if cluster == len(clustering):
            s = set([i])
            clustering.append(s)
          else:
            clustering[cluster].add(i)
          cluster_assn.append(clustering[cluster])
        return clustering, cluster_assn

    def fit(self, data):
        """Run Gibbs sampling to get the cluster assignment """
        num_data = len(data)
        clustering, cluster_assn = self.initialize_assn(data)
        for t in range(self.num_iter):
            num_new_clusters = 0.0
            for i in range(num_data):
                cluster_assn[i].remove(i)
                if len(cluster_assn[i]) == 0:
                    clustering.remove(cluster_assn[i])
                crp_prior = [(len(x) + 0.0) / (num_data - 1 + self.alpha) for j, x in enumerate(clustering)]
                crp_prior.append(self.alpha / (num_data - 1 + self.alpha))
                crp_prior = np.array(crp_prior)
                likelihood = self.likelihood_fn(data, i, clustering, cluster_assn)
                probs = crp_prior * likelihood
                probs = probs/np.sum(probs)

                cluster = np.random.choice(len(probs), p=probs)
                if cluster == len(clustering):
                    s = set([i])
                    clustering.append(s)
                    num_new_clusters += 1
                else:
                    clustering[cluster].add(i)
                cluster_assn[i] = clustering[cluster]
            # Empirical Bayes for adjusting hyperparameters
            if t % self.eb_interval == 0 and t > self.eb_start:
                self.alpha = num_new_clusters
        return clustering, cluster_assn


In [3]:
# 1D gaussian mixture data
true_means = [0, 10, 12]
data = np.concatenate((stats.norm.rvs(0, 1, size=200), stats.norm.rvs(10,1,size=200), stats.norm.rvs(12, 1, size=200)))
random.shuffle(data)

def likelihood_1d(data, i, clustering, cluster_assn):
    means = [np.mean(data[list(cluster)]) for cluster in clustering]
    means.append(0)	
    stds = [1 for cluster in clustering]
    stds.append(10)
    return stats.norm.pdf(data[i], means, stds)

crp_model = CRPClusterModel(1.0, likelihood_1d)
clustering, cluster_assn = crp_model.fit(data)
means = [np.mean(data[list(cluster)]) for cluster in clustering]
print 'True means are %s.\nCluster means are %s.' % (true_means, means)	

True means are [0, 10, 12].
Cluster means are [11.907735515329081, 0.011840096306146276, 9.8749655152559512].


In [5]:
# 2D gaussian mixture model
data = np.concatenate([np.random.randn(100, 2), np.random.randn(100, 2)*0.5+10])
random.shuffle(data)

def likelihood_2d(data, i, clustering, unused_clustering_assn, unknown_std=5.0, unknown_mean=0):
    N, d = data.shape
    means = [np.mean(data[list(cluster)], axis=0) for cluster in clustering]
    means.append(unknown_mean*np.ones(d))
    stds = [np.diag(np.std(data[list(cluster)], axis=0))+(1.0/len(cluster))*np.eye(d) for cluster in clustering]
    stds.append(unknown_std*np.eye(d))
    return [stats.multivariate_normal.pdf(data[i], means[j], stds[j]) for j in range(len(means))]

crp_model = CRPClusterModel(1.0, likelihood_2d)
clustering, cluster_assn = crp_model.fit(data)
means = [np.mean(data[list(cluster)], axis=0) for cluster in clustering]
stds = [np.std(data[list(cluster)], axis=0) for cluster in clustering]

print 'means, stds:'
for mean, std in zip(means, stds):
    print mean, std

means, stds:
[-0.20053728  0.15206846] [ 0.81784412  0.99405943]
[ 10.13685735   9.99394064] [ 0.36567657  0.57149761]
